In [1]:
# code block to track CPU time and Memory usage
import psutil
import time

start_memory = psutil.virtual_memory().available
start_time = time.time()

# 1. COVID-19 graph embedding alone

Load packages

In [2]:
import numpy as np
import pandas as pd
import time
import re
import math
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.modules import Module
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils import add_remaining_self_loops, add_self_loops
from torch_geometric.utils import to_undirected
from torch_geometric.nn import GCNConv, SAGEConv,GAE, VGAE

In [3]:
data_path='data/'
exp_id='v0'
device_id='cpu' #'cpu' if CPU, device number if GPU
embedding_size=128

Load preprocessed files

In [4]:
le=pickle.load(open(data_path+'LabelEncoder_'+exp_id+'.pkl', 'rb'))
edge_index=pickle.load(open(data_path+'edge_index_'+exp_id+'.pkl','rb'))
node_feature_alone_np=pickle.load(open(data_path+'dirichlet_node_feature_'+exp_id+'.pkl','rb'))

In [5]:
node_feature_alone=torch.tensor(node_feature_alone_np, dtype=torch.float)

In [6]:
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)

In [7]:
edge_attr_dict={'gene-drug':0,'gene-gene':1,'bait-gene':2, 'gene-phenotype':3, 'drug-phenotype':4}
edge_index['type']=edge_index['type'].apply(lambda x: edge_attr_dict[x])

In [8]:
edge_index['type'].value_counts()

0    27590
1     5970
3     1618
4     1338
2      247
Name: type, dtype: int64

In [9]:
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)

In [10]:
data = Data(x=node_feature_alone,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr
           )

In [11]:
data.num_features, data.num_nodes,data.num_edges

(400, 15444, 36763)

In [12]:
data.contains_isolated_nodes(), data.is_directed()

/Users/puzhouwang/opt/anaconda3/envs/drg/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)


(True, True)

In [13]:
def train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1):
    r"""Splits the edges of a :obj:`torch_geometric.data.Data` object
    into positive and negative train/val/test edges, and adds attributes of
    `train_pos_edge_index`, `train_neg_adj_mask`, `val_pos_edge_index`,
    `val_neg_edge_index`, `test_pos_edge_index`, and `test_neg_edge_index`
    to :attr:`data`.

    Args:
        data (Data): The data object.
        val_ratio (float, optional): The ratio of positive validation
            edges. (default: :obj:`0.05`)
        test_ratio (float, optional): The ratio of positive test
            edges. (default: :obj:`0.1`)

    :rtype: :class:`torch_geometric.data.Data`
    """

    assert 'batch' not in data  # No batch-mode.

    num_nodes = data.num_nodes
    row, col = data.edge_index
    #data.edge_index = None
    attr = data.edge_attr

    # Return upper triangular portion.
    #mask = row < col
    #row, col = row[mask], col[mask]

    n_v = int(math.floor(val_ratio * row.size(0)))
    n_t = int(math.floor(test_ratio * row.size(0)))

    # Positive edges.
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]
    attr=attr[perm]

    r, c = row[:n_v], col[:n_v]
    data.val_pos_edge_index = torch.stack([r, c], dim=0)
    data.val_pos_edge_attr = attr[:n_v]
    
    r, c = row[n_v:n_v + n_t], col[n_v:n_v + n_t]
    data.test_pos_edge_index = torch.stack([r, c], dim=0)
    data.test_post_edge_attr = attr[n_v:n_v + n_t]

    r, c = row[n_v + n_t:], col[n_v + n_t:]
    data.train_pos_edge_index = torch.stack([r, c], dim=0)
    data.train_pos_edge_attr = attr[n_v+n_t:]

    # Negative edges.
    neg_adj_mask = torch.ones(num_nodes, num_nodes, dtype=torch.uint8)
    neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
    neg_adj_mask[row, col] = 0

    neg_row, neg_col = neg_adj_mask.nonzero().t()
    perm = random.sample(range(neg_row.size(0)),
                         min(n_v + n_t, neg_row.size(0)))
    perm = torch.tensor(perm)
    perm = perm.to(torch.long)
    neg_row, neg_col = neg_row[perm], neg_col[perm]

    neg_adj_mask[neg_row, neg_col] = 0
    data.train_neg_adj_mask = neg_adj_mask

    row, col = neg_row[:n_v], neg_col[:n_v]
    data.val_neg_edge_index = torch.stack([row, col], dim=0)

    row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
    data.test_neg_edge_index = torch.stack([row, col], dim=0)

    return data

In [14]:
device=torch.device(device_id)

In [15]:
data_split=train_test_split_edges(data, test_ratio=0.1, val_ratio=0)
x, train_pos_edge_index, train_pos_edge_attr = data_split.x.to(device), \
    data_split.train_pos_edge_index.to(device), data_split.train_pos_edge_attr.to(device)


In [16]:
train_pos_edge_index, train_pos_edge_attr=add_remaining_self_loops(train_pos_edge_index,train_pos_edge_attr)

In [17]:
pd.Series(train_pos_edge_attr.cpu().numpy()).value_counts()

0    24816
1    20760
3     1461
4     1223
2      221
dtype: int64

In [18]:
x,train_pos_edge_index,train_pos_edge_attr = Variable(x),Variable(train_pos_edge_index),Variable(train_pos_edge_attr)

Define VGAE model

In [19]:
class Encoder_VGAE(nn.Module):
    def __init__(self, in_channels, out_channels, isClassificationTask=False):
        super(Encoder_VGAE, self).__init__()
        self.isClassificationTask=isClassificationTask
        self.conv_gene_drug=  SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_gene = SAGEConv(in_channels, 2*out_channels, )
        self.conv_bait_gene = SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_phenotype = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_phenotype = SAGEConv(in_channels, 2*out_channels)

        self.bn = nn.BatchNorm1d(5*2*out_channels)
        #variational encoder
        self.conv_mu = SAGEConv(5*2*out_channels, out_channels, )
        self.conv_logvar = SAGEConv(5*2*out_channels, out_channels,)

    def forward(self,x,edge_index,edge_attr):
        
        x = F.dropout(x, training=self.training)
        
        index_gene_drug=(edge_attr==0).nonzero().reshape(1,-1)[0]
        edge_index_gene_drug=edge_index[:, index_gene_drug]
        
        index_gene_gene=(edge_attr==1).nonzero().reshape(1,-1)[0]
        edge_index_gene_gene=edge_index[:, index_gene_gene]
        
        index_bait_gene=(edge_attr==2).nonzero().reshape(1,-1)[0]
        edge_index_bait_gene=edge_index[:, index_bait_gene]
        
        index_gene_phenotype=(edge_attr==3).nonzero().reshape(1,-1)[0]
        edge_index_gene_phenotype=edge_index[:, index_gene_phenotype]
        
        index_drug_phenotype=(edge_attr==4).nonzero().reshape(1,-1)[0]
        edge_index_drug_phenotype=edge_index[:, index_drug_phenotype]
        
        
        x_gene_drug = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_gene_drug)), p=0.5, training=self.training, )
        x_gene_gene = F.dropout(F.relu(self.conv_gene_gene(x,edge_index_gene_gene)), p=0.5, training=self.training)
        x_bait_gene = F.dropout(F.relu(self.conv_bait_gene(x,edge_index_bait_gene)), p=0.1, training=self.training)
        x_gene_phenotype = F.dropout(F.relu(self.conv_gene_phenotype(x,edge_index_gene_phenotype)), training=self.training)
        x_drug_phenotype = F.dropout(F.relu(self.conv_drug_phenotype(x,edge_index_drug_phenotype)), training=self.training)

        x=self.bn(torch.cat([x_gene_drug,x_gene_gene,x_bait_gene,x_gene_phenotype,x_drug_phenotype],dim=1))        
        
        return self.conv_mu(x,edge_index), self.conv_logvar(x,edge_index)

In [20]:
model=VGAE(Encoder_VGAE(node_feature_alone.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [21]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index, train_pos_edge_attr)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    print(loss.item())
    
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z=model.encode(x, train_pos_edge_index,train_pos_edge_attr)
    return model.test(z, pos_edge_index, neg_edge_index)

In [22]:
# baseline
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index )

(0.35108383971080825, 0.40174833765204654)

In [23]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


15.416215896606445
Epoch: 001, AUC: 0.5256, AP: 0.4825
15.016587257385254
Epoch: 002, AUC: 0.5133, AP: 0.4709
14.950262069702148
Epoch: 003, AUC: 0.5184, AP: 0.4718
14.580504417419434
Epoch: 004, AUC: 0.5466, AP: 0.4879
14.047994613647461
Epoch: 005, AUC: 0.5889, AP: 0.5173
13.703048706054688
Epoch: 006, AUC: 0.6673, AP: 0.6018
13.098477363586426
Epoch: 007, AUC: 0.7366, AP: 0.7173
12.302462577819824
Epoch: 008, AUC: 0.7613, AP: 0.7618
11.557718276977539
Epoch: 009, AUC: 0.7730, AP: 0.7840


Node embedding

In [24]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

Save the new embedding 

In [25]:
pickle.dump(z_np, open(data_path+'COVID_embedding_'+exp_id+'.pkl', 'wb'))

Save the torch model

In [26]:
torch.save(model.state_dict(), data_path+'COVID_VAE_encoders_'+exp_id+'.pkl')

In [27]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
start_time = time.time()
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
start_memory = end_memory
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')

CPU time: 95.41 sec
Memory usage: 169.02 MB


# 2. Hybrid Embedding: COVID-19 graph + DRKG pre-train

## 2.1. embedding dimension: 128 (default)

In [28]:
embedding_size = 128
node_feature_np=pickle.load(open(data_path+'node_feature_'+exp_id+'.pkl','rb'))
node_feature=torch.tensor(node_feature_np, dtype=torch.float)

In [29]:
data = Data(x=node_feature,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr
           )

In [30]:
data_split=train_test_split_edges(data, test_ratio=0.1, val_ratio=0)
x, train_pos_edge_index, train_pos_edge_attr = data_split.x.to(device), \
    data_split.train_pos_edge_index.to(device), data_split.train_pos_edge_attr.to(device)

In [31]:
train_pos_edge_index, train_pos_edge_attr=add_remaining_self_loops(train_pos_edge_index,train_pos_edge_attr)

In [32]:
x,train_pos_edge_index,train_pos_edge_attr = Variable(x),Variable(train_pos_edge_index),Variable(train_pos_edge_attr)

In [33]:
model=VGAE(Encoder_VGAE(node_feature.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [34]:
#DRKG's accuracy for comparison
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index)

(0.3724476384772681, 0.4820759211539945)

In [35]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

19.13382339477539
Epoch: 001, AUC: 0.6024, AP: 0.5274
17.525449752807617
Epoch: 002, AUC: 0.6308, AP: 0.5383
16.857994079589844
Epoch: 003, AUC: 0.6638, AP: 0.5734
16.213788986206055
Epoch: 004, AUC: 0.6923, AP: 0.6242
16.46436309814453
Epoch: 005, AUC: 0.7115, AP: 0.6518
16.6589298248291
Epoch: 006, AUC: 0.7281, AP: 0.6715
16.616436004638672
Epoch: 007, AUC: 0.7574, AP: 0.7056
16.045394897460938
Epoch: 008, AUC: 0.7878, AP: 0.7436
15.488917350769043
Epoch: 009, AUC: 0.7969, AP: 0.7370


In [36]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

In [37]:
pickle.dump(z_np, open(data_path+f'hybrid_embedding_{embedding_size}_'+exp_id+'.pkl', 'wb'))

In [38]:
torch.save(model.state_dict(), data_path+f'hybrid_VAE_encoders_{embedding_size}_'+exp_id+'.pkl')

In [39]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
start_time = time.time()
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
start_memory = end_memory
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')

CPU time: 84.95 sec
Memory usage: 111.70 MB


## 2.2. embedding dimension: 64

In [40]:
embedding_size = 64
model=VGAE(Encoder_VGAE(node_feature.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [41]:
#DRKG's accuracy for comparison
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index)

(0.3724476384772681, 0.4820759211539945)

In [42]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

15.455953598022461
Epoch: 001, AUC: 0.6181, AP: 0.5785
13.9932222366333
Epoch: 002, AUC: 0.6777, AP: 0.6492
13.187482833862305
Epoch: 003, AUC: 0.7203, AP: 0.7233
12.447782516479492
Epoch: 004, AUC: 0.7449, AP: 0.7589
11.468147277832031
Epoch: 005, AUC: 0.7607, AP: 0.7715
11.030091285705566
Epoch: 006, AUC: 0.7695, AP: 0.7773
10.51605224609375
Epoch: 007, AUC: 0.7649, AP: 0.7722
9.613448143005371
Epoch: 008, AUC: 0.7478, AP: 0.7580
9.524665832519531
Epoch: 009, AUC: 0.7478, AP: 0.7558


In [43]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

In [44]:
pickle.dump(z_np, open(data_path+f'hybrid_embedding_{embedding_size}_'+exp_id+'.pkl', 'wb'))

In [45]:
torch.save(model.state_dict(), data_path+f'hybrid_VAE_encoders_{embedding_size}_'+exp_id+'.pkl')

In [46]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
start_time = time.time()
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
start_memory = end_memory
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')

CPU time: 28.31 sec
Memory usage: -0.15 MB


## 2.3. embedding dimension: 256

In [47]:
embedding_size = 256
model=VGAE(Encoder_VGAE(node_feature.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [48]:
#DRKG's accuracy for comparison
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index)

(0.3724476384772681, 0.4820759211539945)

In [49]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

21.65967559814453
Epoch: 001, AUC: 0.5596, AP: 0.4878
18.980850219726562
Epoch: 002, AUC: 0.6474, AP: 0.5853
18.603179931640625
Epoch: 003, AUC: 0.6624, AP: 0.5999
19.17207908630371
Epoch: 004, AUC: 0.6681, AP: 0.6055
19.3270263671875
Epoch: 005, AUC: 0.6629, AP: 0.6018
18.969833374023438
Epoch: 006, AUC: 0.6737, AP: 0.6108
18.517654418945312
Epoch: 007, AUC: 0.6928, AP: 0.6265
17.79478645324707
Epoch: 008, AUC: 0.7123, AP: 0.6433
17.227054595947266
Epoch: 009, AUC: 0.7274, AP: 0.6564


In [50]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

In [51]:
pickle.dump(z_np, open(data_path+f'hybrid_embedding_{embedding_size}_'+exp_id+'.pkl', 'wb'))

In [52]:
torch.save(model.state_dict(), data_path+f'hybrid_VAE_encoders_{embedding_size}_'+exp_id+'.pkl')

In [53]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
start_time = time.time()
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
start_memory = end_memory
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')

CPU time: 184.06 sec
Memory usage: 31.23 MB


## 2.4. default embedding size (128) without bait-prey information

In [54]:
edge_index = pickle.load(open(data_path+'edge_index_no_bp_'+exp_id+'.pkl','rb'))
node_feature_no_bp_np = pickle.load(open(data_path+'node_feature_no_bp_'+exp_id+'.pkl','rb'))

In [55]:
node_feature_no_bp = torch.tensor(node_feature_no_bp_np, dtype=torch.float)

In [56]:
edge=torch.tensor(edge_index[['node1', 'node2']].values, dtype=torch.long)

In [57]:
edge_attr_dict={'gene-drug':0,'gene-gene':1,'gene-phenotype':3, 'drug-phenotype':4}
edge_index['type']=edge_index['type'].apply(lambda x: edge_attr_dict[x])

In [58]:
edge_index['type'].value_counts()

0    27590
1     5970
3     1618
4     1338
Name: type, dtype: int64

In [59]:
edge_attr=torch.tensor(edge_index['type'].values,dtype=torch.long)

In [60]:
data = Data(x=node_feature_no_bp,
            edge_index=edge.t().contiguous(),
            edge_attr=edge_attr
           )

In [61]:
data.num_features, data.num_nodes,data.num_edges

(400, 15444, 36516)

In [62]:
data_split=train_test_split_edges(data, test_ratio=0.1, val_ratio=0)
x, train_pos_edge_index, train_pos_edge_attr = data_split.x.to(device), \
    data_split.train_pos_edge_index.to(device), data_split.train_pos_edge_attr.to(device)


In [63]:
train_pos_edge_index, train_pos_edge_attr=add_remaining_self_loops(train_pos_edge_index,train_pos_edge_attr)

In [64]:
x,train_pos_edge_index,train_pos_edge_attr = Variable(x),Variable(train_pos_edge_index),Variable(train_pos_edge_attr)

In [65]:
class Encoder_VGAE_no_bp(nn.Module):
    def __init__(self, in_channels, out_channels, isClassificationTask=False):
        super(Encoder_VGAE_no_bp, self).__init__()
        self.isClassificationTask=isClassificationTask
        self.conv_gene_drug=  SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_gene = SAGEConv(in_channels, 2*out_channels, )
        self.conv_gene_phenotype = SAGEConv(in_channels, 2*out_channels, )
        self.conv_drug_phenotype = SAGEConv(in_channels, 2*out_channels)

        self.bn = nn.BatchNorm1d(4*2*out_channels)
        #variational encoder
        self.conv_mu = SAGEConv(4*2*out_channels, out_channels, )
        self.conv_logvar = SAGEConv(4*2*out_channels, out_channels,)

    def forward(self,x,edge_index,edge_attr):
        
        x = F.dropout(x, training=self.training)
        
        index_gene_drug=(edge_attr==0).nonzero().reshape(1,-1)[0]
        edge_index_gene_drug=edge_index[:, index_gene_drug]
        
        index_gene_gene=(edge_attr==1).nonzero().reshape(1,-1)[0]
        edge_index_gene_gene=edge_index[:, index_gene_gene]
        
        index_gene_phenotype=(edge_attr==3).nonzero().reshape(1,-1)[0]
        edge_index_gene_phenotype=edge_index[:, index_gene_phenotype]
        
        index_drug_phenotype=(edge_attr==4).nonzero().reshape(1,-1)[0]
        edge_index_drug_phenotype=edge_index[:, index_drug_phenotype]
        
        
        x_gene_drug = F.dropout(F.relu(self.conv_gene_drug(x,edge_index_gene_drug)), p=0.5, training=self.training, )
        x_gene_gene = F.dropout(F.relu(self.conv_gene_gene(x,edge_index_gene_gene)), p=0.5, training=self.training)
        x_gene_phenotype = F.dropout(F.relu(self.conv_gene_phenotype(x,edge_index_gene_phenotype)), training=self.training)
        x_drug_phenotype = F.dropout(F.relu(self.conv_drug_phenotype(x,edge_index_drug_phenotype)), training=self.training)

        x=self.bn(torch.cat([x_gene_drug,x_gene_gene,x_gene_phenotype,x_drug_phenotype],dim=1))        
        
        return self.conv_mu(x,edge_index), self.conv_logvar(x,edge_index)

In [66]:
embedding_size = 128
model=VGAE(Encoder_VGAE_no_bp(node_feature.shape[1], embedding_size)).to(device)
optimizer=torch.optim.Adam(model.parameters())

In [67]:
#DRKG's accuracy for comparison
model.test(x,data_split.test_pos_edge_index, data_split.test_neg_edge_index)

(0.39977464779856814, 0.45464179100308316)

In [68]:
for epoch in range(1, 10):
    train()
    auc, ap = test(data_split.test_pos_edge_index, data_split.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

18.5837345123291
Epoch: 001, AUC: 0.4963, AP: 0.4535
18.44424819946289
Epoch: 002, AUC: 0.5180, AP: 0.4612
18.143022537231445
Epoch: 003, AUC: 0.5501, AP: 0.4767
17.294086456298828
Epoch: 004, AUC: 0.5897, AP: 0.5000
17.259063720703125
Epoch: 005, AUC: 0.6296, AP: 0.5308
15.941333770751953
Epoch: 006, AUC: 0.6643, AP: 0.5784
15.271269798278809
Epoch: 007, AUC: 0.6825, AP: 0.6293
14.651122093200684
Epoch: 008, AUC: 0.6836, AP: 0.6464
14.083636283874512
Epoch: 009, AUC: 0.6823, AP: 0.6522


In [69]:
model.eval()
z=model.encode(x, data.edge_index.to(device), data.edge_attr.to(device))
z_np = z.squeeze().detach().cpu().numpy()

In [70]:
pickle.dump(z_np, open(data_path+f'hybrid_embedding_{embedding_size}_no_bp_'+exp_id+'.pkl', 'wb'))

In [71]:
torch.save(model.state_dict(), data_path+f'hybrid_VAE_encoders_{embedding_size}_no_bp_'+exp_id+'.pkl')

In [72]:
# code block to track CPU / Memory usage
cpu_sec = time.time() - start_time
start_time = time.time()
end_memory = psutil.virtual_memory().available
mem_use = (start_memory - end_memory) / (1024.0 ** 2)
start_memory = end_memory
print(f'CPU time: {cpu_sec:.2f} sec')
print(f'Memory usage: {mem_use:.2f} MB')

CPU time: 68.85 sec
Memory usage: -49.72 MB
